In [1]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from statsmodels.formula.api import ols, glm
import statsmodels.api as sm

In [2]:
df = pd.read_parquet('../data/curated/sample_data_remove_outliers.parquet')

In [3]:
#Since there is tip_amount = 0, log transformation would have problem with it
log_transformed_data = np.log(df.loc[df['tip_amount'] > 0,'tip_amount'])

In [4]:
filtered_df = df[df['tip_amount'] != 0]
log_transformed_data = np.log(filtered_df['tip_amount'])

In [5]:
#Transforming those numeric datatype to categorical to prepare f test
filtered_df['PULocationID'] = pd.Categorical(filtered_df['PULocationID'])
filtered_df['DOLocationID'] = pd.Categorical(filtered_df['DOLocationID'])

/var/folders/d9/jcb14qr94sg1lztfzpxkm77r0000gn/T/ipykernel_31925/2240414701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PULocationID'] = pd.Categorical(filtered_df['PULocationID'])
/var/folders/d9/jcb14qr94sg1lztfzpxkm77r0000gn/T/ipykernel_31925/2240414701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['DOLocationID'] = pd.Categorical(filtered_df['DOLocationID'])


## ANOVA test ##

In [6]:
# Create an ANOVA model
model = ols('log_transformed_data ~ Weekend+Airport+Congestion+PULocationID+DOLocationID', data=filtered_df).fit()

# Perform the ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table) 


                    sum_sq        df            F        PR(>F)
Weekend          74.797129       1.0   236.656787  2.323348e-53
Airport          74.239905       1.0   234.893739  5.622621e-53
Congestion     1663.384381       1.0  5262.918614  0.000000e+00
PULocationID   2147.208404     154.0    44.115125  0.000000e+00
DOLocationID   9174.217908     242.0   119.946539  0.000000e+00
Residual      45640.585104  144406.0          NaN           NaN
